# Assignment 10

### Q1. What is the difference between `__getattr__` and `__getattribute__` ?

ANSWER : Both `__getattr__` and `__getattribute__` are special methods in Python used to handle attribute access in classes, but they differ in how they are invoked and how they handle attribute access.

`__getattr__` is called when an attribute is not found in the usual places, such as the instance dictionary or the class hierarchy;  It’s good for implementing a fallback for missing attributes. It takes a single argument, the name of the attribute being accessed, and should return the value of the attribute or raise an `AttributeError` if the attribute is not found.

In [1]:
class Name:
    def __getattr__(self, name):
        print(f"{name} not found")
        return None

obj = Name()
print(obj.some_name)  # prints "some_name not found" and returns None

some_name not found
None


`__getattribute__`, is called for every attribute access, regardless of whether the attribute exists or not. It takes a single argument, the name of the attribute being accessed, and should return the value of the attribute or raise an `AttributeError` if the attribute is not found.

`__getattribute__` is used to customize the behavior of attribute access for all attributes of an object and is invoked before looking at the actual attributes on the object, and so can be tricky to implement correctly. One can end up in infinite recursions very easily.

Note : It's important to use `super().__getattribute__(name)` in the implementation of `__getattribute__` to avoid infinite recursion. If you attempt to access an attribute within `__getattribute__` using self.name, it will trigger another call to `__getattribute__`, leading to an infinite loop.

In [2]:
class MyClass:
    def __init__(self):
        self.x = 1
        self.y = 2.3

    def __getattribute__(self, name):
        value = super().__getattribute__(name)
        if isinstance(value, int):
            print(f'Accessing the value attribute: {name}')
            return value * 10
        else:
            print(f'Accessing the value attribute: {name}')
            return value

obj = MyClass()
print(obj.x)
print(obj.y)

Accessing the value attribute: x
10
Accessing the value attribute: y
2.3


In [3]:
# Another Example:

class Name:
    def __getattr__(self, name):
        print(f"{name} not found")
        return None
    
    def __getattribute__(self, name):
        print(f"Accessing attribute : {name}")
        return super().__getattribute__(name)
#         return object.__getattribute__(self, name)

obj = Name()
obj.some_name = "rahul"
print(obj.some_name) 

Accessing attribute : some_name
rahul


### Q2. What is the difference between properties and descriptors?

ANSWER : 
#### Property
The property protocol allows us to route a specific attribute’s get and set operations to functions or methods we provide, enabling us to insert code to be run automatically on attribute access, intercept attribute deletions, and provide documentation for the attributes if desired.

A property manages a single, specific attribute; although it can’t catch all attribute accesses generically, it allows us to control both fetch and assignment accesses and enables us to change an attribute from simple data to a computation freely, without breaking existing code.

In [9]:
class Employee:
    def __init__(self, name):
        self._name = name

    def getname(self):                 # name = property(name)
        return self._name

    def setname(self, value):          # name = name.setter(name)
        print(f'Setting Name = {value}')
        self._name = value

    def delname(self):                 # name = name.deleter(name)
        print('Deleting Name')
        del self._name

    name = property( fget=getname, fset=setname, fdel=delname, doc="doc related to Employee class")

In [10]:
obj = Employee('Shiv')

In [11]:
# Accessing the getname() function using `name` 
obj.name

'Shiv'

In [12]:
# Accessing setname() function using `name`
obj.name = 'Krishna'

Setting Name = Krishna


In [13]:
obj.name

'Krishna'

In [14]:
# NOTE: Property can also be set using decoraters
class EMPLOYEE:
    def __init__(self, name):
        self._name = name

    @property
    def name(self):                          # name = property(name)
        return  self._name

    @name.setter
    def name(self, value):                # name = name.setter(name)
        print('setting name...')
        self._name = value

    @name.deleter
    def name(self):                      # name = name.deleter(name)
        print('Deleting Name...')
        del self._name

#### Descriptors
Descriptors provide an alternative way to intercept attribute access; if any class has `__get__()`, `__set__()`, and `__delete__()` functions in it, whose purpose is to allow us to customise what it means to get, set or delete an attribute; such a class is qualified to be called as descriptor.

Since descriptors are coded as normal classes, they have their own state, may participate in descriptor inheritance hierarchies, can use composition to aggregate objects, and provide a natural structure for coding internal methods and attribute documentation strings.


In [16]:
class Celcius:
    def __get__(self, instance, owner):
        print(owner)
        print(instance)
        return instance.kelvin - 273
        
    def __set__(self, instance, value) -> None:
        instance.kelvin = value + 273
        
    def __delete__(self, instance):
        del instance.
    
class Kelvin:
    def __init__(self, kelvin) -> None:
        self.kelvin = kelvin
        
    celcius = Celcius()
        
temp = Kelvin(300)
print(temp.celcius)

<class '__main__.Kelvin'>
27


Parameters passed in the  `__get__` function defined in `Celcius` class:
* `self` : it's `Celcius` class instance.
* `instance` : it's `Kelvin` class instance.
* `owner` : it's `Kelvin` class.

### Q3. What are the key differences in functionality between `__getattr__` and `__getattribute__`, as well as properties and descriptors?

ANSWER : 
#### Differences in functionality between `__getattr__` and `__getattribute__` : 
* `__getattr__(self, name)` is called when an attribute is not found in the usual places, such as the instance's namespace, class hierarchy, or built-ins. It receives the name of the attribute as a string and should return the value of the attribute or raise an AttributeError if the attribute is not found. It is a fallback method for attribute lookup.

* `__getattribute__(self, name)` is called for every attribute access, regardless of whether the attribute exists in the instance's namespace or not. It receives the name of the attribute as a string and should return the value of the attribute. If it needs to access the same attribute it is currently getting, it should use the super() function to avoid an infinite recursion. It is a general hook for attribute access.

#### Differences in functionality between `properties` and `descriptors` :
* Properties are a simple way to define read-only or read-write attributes that behave like normal instance variables, but with custom behavior for getting and/or setting the attribute value.

* Descriptors are a more powerful and general-purpose mechanism for customizing attribute access. They allow you to define custom behavior not only for getting and setting attribute values, but also for other operations such as deleting or setting attribute values with a specific type or format, and a seperate class is defined for desriptors.